<a href="https://colab.research.google.com/github/rajaramkuberan/ANN_Coursework_Coventry_Univ/blob/main/ANN_Coursework_VGG_16_Ashish_Solanki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ANN Coursework**

#### Ashish Solanki

Here, we will train a VGG-16 NN to perform image classification on Ketogenic and Non-Ketogenic Food items, using transfer learning. Transfer learning means we will use. We will be using TensorFlow/Keras to build our neural network and training/test dataset. First, let's download the training and test data:
we have provided dada in github for public use. 

In [ ]:
!git clone https://github.com/rajaramkuberan/ANN_Coursework_Coventry_Univ.git

Cloning into 'ANN_Coursework_Coventry_Univ'...
remote: Enumerating objects: 4697, done.
remote: Counting objects: 100% (4697/4697), done.
remote: Compressing objects: 100% (4689/4689), done.
remote: Total 30301 (delta 8), reused 4697 (delta 8), pack-reused 25604
Receiving objects: 100% (30301/30301), 6.11 GiB | 27.01 MiB/s, done.
Resolving deltas: 100% (305/305), done.
Checking out files: 100% (31400/31400), done.


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, AveragePooling2D, Flatten, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from keras.layers import concatenate
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.metrics import roc_auc_score

#### **Data Preprocessing**

Now ,let's use ImageDataGenerator class to create our train and test dataset and normalize our data. 

It's important to normalize our data because data going into our CNN to improve its overall performance. We will use the rescale parameter to scale our image pixel values from [0, 255] to [0,1].

In each generator, we specify the source directory of our images, the classes, the input image size, the batch size (how many images to process at once), and class mode.

In [ ]:
# initialising the image size as 224x224
IMAGE_SIZE = [224,224]


# Setting the Path to test and train data 
test_path = '/content/ANN_Coursework_Coventry_Univ/test'
train_path = '/content/ANN_Coursework_Coventry_Univ/train'

In [ ]:
# transforming train data images
train_datagen = ImageDataGenerator(rescale = 1/255)

# transforming train data images
test_datagen = ImageDataGenerator(rescale = 1/255)


In [ ]:
#training data 
train_generator = train_datagen.flow_from_directory(
        '/content/ANN_Coursework_Coventry_Univ/train',  # This is the source directory for training images
        classes = ['Keto_train', 'Non_Keto_train'],
        target_size=(224, 224),  # All images will be resized to 224x224
        batch_size=32,
        # Use binary labels
        class_mode='binary')

# testing data
test_generator = test_datagen.flow_from_directory(
        '/content/ANN_Coursework_Coventry_Univ/test',  # This is the source directory for training images
        classes = ['Keto_test', 'Non_Keto_Test'],
        target_size=(224, 224),  # All images will be resized to 224x224
        batch_size=16,
        # Use binary labels
        class_mode='binary')

Found 24861 images belonging to 2 classes.
Found 6163 images belonging to 2 classes.


# **Building the Model**

But before we continue, let's start defining the model:


In [ ]:
model = Sequential()
model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))


model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))


model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))


model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))


model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))


model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=2, activation="softmax"))


The model.summary() method call prints a summary of the NN 

In [ ]:
#model.summary()

The "output shape" column shows the transformation of the dimensions of each layer as a result of the convolution and max pooling - convolution will reduce the layer size by a bit due to padding, and max pooling will halve the output size.


Next, we'll configure the specifications for model training. We will train our model with the binary_crossentropy loss. We will use the Adam optimizer with a learning rate of 0.001. Adam is a sensible optimization algorithm because it automates learning-rate tuning for us (alternatively, we could also use Rms and Adagrad for similar results). We will add accuracy to metrics so that the model will monitor accuracy during training

In [ ]:
from tensorflow.keras.optimizers import Adam


from tensorflow.keras import optimizers


model.compile(optimizer='Adam', loss = 'sparse_categorical_crossentropy', metrics = 'accuracy', )

### **Training**
Let's train for 15 epochs -- this may take a few minutes to run.

Note that steps_per_epoch was set along with batch_size in ImageDataGenerator.

Notice that as we train, our test accuracy never exceeds training accuracy, which is a good thing. Our model won't work better on unseen images than seen images.

In [ ]:
# epochs is 30  
history = model.fit(
      train_generator,
      steps_per_epoch=1,  
      epochs=2,
      verbose=1,
      validation_data = test_generator,
      validation_steps=1)

# Accuracy, ROC Curve, and AUC




In [ ]:
model.evaluate(test_generator)

386/386 [==============================] - 70s 181ms/step - loss: 0.6543 - accuracy: 0.6393


[0.6542653441429138, 0.6392990350723267]

In [ ]:
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
test_generator.reset()
preds = model.predict(test_generator,
                      verbose=1)

In [ ]:
fpr, tpr, _ = roc_curve(test_generator.classes, preds)

In [ ]:
roc_auc = auc(fpr, tpr)

In [ ]:
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=1, mode='auto')

In [ ]:
hist = model.fit_generator(steps_per_epoch=10,generator=train_generator, validation_data= test_generator, validation_steps=10,epochs=10,callbacks=[checkpoint,early])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
10/10 [==============================] - 7s 668ms/step - loss: 0.6639 - accuracy: 0.6250 - val_loss: 0.6583 - val_accuracy: 0.6313
Epoch 2/10
10/10 [==============================] - 7s 656ms/step - loss: 0.6677 - accuracy: 0.6125 - val_loss: 0.6704 - val_accuracy: 0.6062
Epoch 3/10
10/10 [==============================] - 6s 642ms/step - loss: 0.6454 - accuracy: 0.6625 - val_loss: 0.6585 - val_accuracy: 0.6313
Epoch 4/10
10/10 [==============================] - 7s 658ms/step - loss: 0.6531 - accuracy: 0.6406 - val_loss: 0.6702 - val_accuracy: 0.6125
Epoch 5/10
10/10 [==============================] - 7s 677ms/step - loss: 0.6618 - accuracy: 0.6281 - val_loss: 0.6662 - val_accuracy: 0.6187
Epoch 6/10
10/10 [==============================] - 6s 640ms/step - loss: 0.6714 - accuracy: 0.6062 - val_loss: 0.6647 - val_accuracy: 0.6187
Epoch 7/10
10/10 [==============================] - 7s 648ms/step - loss: 0.6390 - accuracy: 0.6719 - val_loss: 0.6519 - val_accuracy: 0.6438
Epoch 

In [ ]:
import matplotlib.pyplot as plt
plt.plot(hist.history["acc"])
plt.plot(hist.history['val_acc'])
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title("model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy","Validation Accuracy","loss","Validation Loss"])
plt.show()

In [ ]:
from keras.preprocessing import image
img = image.load_img("/content/1.jpg",target_size=(224,224))
img = np.asarray(img)
plt.imshow(img)
img = np.expand_dims(img, axis=0)
from keras.models import load_model
saved_model = load_model("vgg16_1.h5")
output = saved_model.predict(img)
if output[0][0] > output[0][1]:
    print("keto")
else:
    print('non-keto')